<a href="https://colab.research.google.com/github/JavierBescosArtigas/MBID08Visualization-GuidedActies/blob/main/MBID08PlotlyJavierBescos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pycountry_convert 
from google.colab import drive
drive.mount("/content/drive")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.1 MB 6.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 210 kB 46.0 MB/s 
     |████████████████████████████████| 316 kB 11.9 MB/s 
     |████████████████████████████████| 298 kB 41.6 MB/s 
     |████████████████████████████████| 297 kB 46.4 MB/s 
     |████████████████████████████████| 297 kB 51.7 MB/s 
     |████████████████████████████████| 297 kB 48.8 MB/s 
     |████████████████████████████████| 296 kB 50.7 MB/s 
     |████████████████████████████████| 296 kB 42.1 MB/s 
     |████████████████████████████████| 280 kB 39.1 MB/s 
     |████████████████████████████████| 280 kB 45.5 MB/s 
     |████████████████████████████████| 280 kB 35.2 MB/s 
     |████████████████████████████████| 280 kB 51.9 MB/s 
     |███████████████

# ¿Vivimos en 1 mundo más violento?
* Macro ~ parece que si, o que no, o que estamos a merced de otros
* Micro ~ muertes violentas, se trabajará con excel y/o datawraper --> [link](https://datos.bancomundial.org/indicador/VC.IHR.PSRC.P5?view=chart)


### imports y carga de datasets.



In [2]:
import plotly.express as px
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pycountry_convert as pc
import ipywidgets as widgets
import plotly.graph_objects as go

In [3]:
!unzip /content/drive/MyDrive/MBID/gastomilitarpercentPIB.zip 
!head -n 10 /content/API_MS.MIL.XPND.GD.ZS_DS2_es_csv_v2_4523928.csv

Archive:  /content/drive/MyDrive/MBID/gastomilitarpercentPIB.zip
  inflating: Metadata_Indicator_API_MS.MIL.XPND.GD.ZS_DS2_es_csv_v2_4523928.csv  
  inflating: API_MS.MIL.XPND.GD.ZS_DS2_es_csv_v2_4523928.csv  
  inflating: Metadata_Country_API_MS.MIL.XPND.GD.ZS_DS2_es_csv_v2_4523928.csv  
﻿"Data Source","Indicadores del desarrollo mundial",

"Last Updated Date","2022-09-16",

"Country Name","Country Code","Indicator Name","Indicator Code","1960","1961","1962","1963","1964","1965","1966","1967","1968","1969","1970","1971","1972","1973","1974","1975","1976","1977","1978","1979","1980","1981","1982","1983","1984","1985","1986","1987","1988","1989","1990","1991","1992","1993","1994","1995","1996","1997","1998","1999","2000","2001","2002","2003","2004","2005","2006","2007","2008","2009","2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020","2021",
"Aruba","ABW","Gasto militar (% del PIB)","MS.MIL.XPND.GD.ZS","","","","","","","","","","","","","","","","","","","","",

In [4]:
gastoMilitar = pd.read_csv("/content/API_MS.MIL.XPND.GD.ZS_DS2_es_csv_v2_4523928.csv", skiprows = 4) #% del pib
#gastoMilitar = gastoMilitar.drop(gastoMilitar[gastoMilitar['Country Name'].isna()].index )
gastoMilitar = gastoMilitar.drop(columns = ['Unnamed: 66',])

#nueva columna
gastoMilitar=gastoMilitar.assign(Continente = 1) 
def getContinents(x):
  #from alphacode3 in dataframe
    try:
        return  pc.convert_continent_code_to_continent_name(pc.country_alpha2_to_continent_code(pc.country_alpha3_to_country_alpha2(x)) )
    except:
        return 'Undetermined'
gastoMilitar.Continente = gastoMilitar["Country Code"].apply(getContinents)

gastoMilitar = gastoMilitar.drop(gastoMilitar[gastoMilitar['Continente'] == 'Undetermined'].index )

gastoMilitar = gastoMilitar.melt(id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', 'Continente'], var_name='Año').sort_values(['Country Name', 'Año', 'Continente'])
gastoMilitar.rename(columns = {"value": "GastoMilitar%"}, inplace = True)
#NAS with fill foward by country

gastoMilitar['GastoMilitar%'] = gastoMilitar.groupby('Country Name')['GastoMilitar%'].ffill()#ffoward

#gastoMilitar['GastoMilitar%'] = gastoMilitar.groupby('Country Name')['GastoMilitar%'].bfill()#fbackward


display(gastoMilitar[gastoMilitar['GastoMilitar%'].isna()]['GastoMilitar%'].count() , gastoMilitar['GastoMilitar%'].count())
#display(gastoMilitar.isnull().sum()) #nº de nulls

0

8004

### Plot 1 Gasto en defensa sobre porcentaje del PIB desde 1960 separado por 6 continentes


In [5]:
fig1 = px.line(gastoMilitar, x="Año", y="GastoMilitar%", color="Country Name", line_group="Country Name", hover_name="Country Name",
        line_shape="spline", render_mode="svg",facet_col="Continente", range_y=[1,80], title="Gasto en defensa como porcentaje del PIB desde 1960 hasta 2020, separado por continentes",
         labels={
                     "GastoMilitar%": "Gasto Militar % sobre PIB",
                     "Country Name": "Seleccione país de la lista para ocultar/focalizar",
                    "Año": ''
                     
                 },
        )
fig1.update_layout(showlegend=True)
fig1.show()

## Imports y carga dataset 2

In [6]:
!unzip /content/drive/MyDrive/MBID/militaryExpenditureDatosBancoMundial.zip 

Archive:  /content/drive/MyDrive/MBID/militaryExpenditureDatosBancoMundial.zip
  inflating: Metadata_Indicator_API_MS.MIL.XPND.CD_DS2_es_csv_v2_4541472.csv  
  inflating: API_MS.MIL.XPND.CD_DS2_es_csv_v2_4541472.csv  
  inflating: Metadata_Country_API_MS.MIL.XPND.CD_DS2_es_csv_v2_4541472.csv  


In [7]:
gastoMilitarAbsoluto = pd.read_csv("/content/API_MS.MIL.XPND.CD_DS2_es_csv_v2_4541472.csv", skiprows = 4)
gastoMilitarAbsoluto = gastoMilitarAbsoluto.drop(columns = ['Unnamed: 66',])

#nueva columna para homogeneizar qué paises eliminamos
gastoMilitarAbsoluto=gastoMilitarAbsoluto.assign(Continente = 1)

gastoMilitarAbsoluto.Continente = gastoMilitarAbsoluto["Country Code"].apply(getContinents)
gastoMilitarAbsoluto = gastoMilitarAbsoluto.drop(gastoMilitarAbsoluto[gastoMilitarAbsoluto['Continente'] == 'Undetermined'].index )

gastoMilitarAbsoluto = gastoMilitarAbsoluto.melt(id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', 'Continente'], var_name='Año').sort_values(['Country Name', 'Año', 'Continente'])
gastoMilitarAbsoluto.rename(columns = {"value": "GastoMilitarTotal"}, inplace = True)
gastoMilitarAbsoluto['GastoMilitarTotal'] = gastoMilitarAbsoluto.groupby('Country Name')['GastoMilitarTotal'].ffill()#ffoward

gastoMilitarAbsoluto.Continente.unique()
gastoMilitarAbsoluto.columns

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       'Continente', 'Año', 'GastoMilitarTotal'],
      dtype='object')

### Plot 2, Gasto en defensa absoluto (USD) desde 1960 separado por continentes

In [8]:
fig2 = px.line(gastoMilitarAbsoluto, x="Año", y="GastoMilitarTotal", color="Country Name", line_group="Country Name", hover_name="Country Name",
        line_shape="spline", render_mode="svg",facet_col="Continente", title="Gasto en defensa absoluto (USD) desde 1960 hasta 2020, separado por continentes.", range_y=[1,250000000000],
        labels={
                     "GastoMilitarTotal": "Gasto Militar Total (USD)",
                     "Country Name": "Seleccione país de la lista para ocultar/focalizar",
                     'Año' : '',
                     'Continente=': ''
                 }, 
        )
fig2.update_layout(showlegend=True)
fig2.show()


## Vemos que sólo unos pocos agentes internacionales gastan la inmensa mayoría, del gasto. Veámoslo más en detalle.

In [9]:
top15absoluto2020 = gastoMilitarAbsoluto[gastoMilitarAbsoluto['Año'] == '2020']
listaTop15 = list(top15absoluto2020.nlargest(15, ['GastoMilitarTotal'])['Country Name'])
#este diccionario acompañado de la función siguiente nos permitirá etiquetar todos los dataframes para quedarnos con los 15 países más importantes en 2020 ¡pero desde 1960!
dictOrdenDeseado = {}
for i in range(len(listaTop15)):
 dictOrdenDeseado[listaTop15[i]] = i
print(dictOrdenDeseado)
def ObtainOrdenDeseado(x):
  try:
    return(dictOrdenDeseado[x])
  except:
    return -1

#nueva columna
top15absoluto2020 = gastoMilitarAbsoluto.assign(top15_2020 = 0) 
top15absoluto2020['top15_2020'] = top15absoluto2020['Country Name'].apply(ObtainOrdenDeseado)#aquí se toman todos los años

NoTop15absoluto2020 = top15absoluto2020[top15absoluto2020['top15_2020'] == -1  ] #para plot futuro

top15absoluto2020 = top15absoluto2020[top15absoluto2020['top15_2020'] > -1  ]
top15absoluto2020.head()


{'Estados Unidos': 0, 'China': 1, 'India': 2, 'Federación de Rusia': 3, 'Reino Unido': 4, 'Arabia Saudita': 5, 'Alemania': 6, 'Francia': 7, 'Japón': 8, 'Corea, República de': 9, 'Italia': 10, 'Australia': 11, 'Emiratos Árabes Unidos': 12, 'Canadá': 13, 'Israel': 14}


,Country Name,Country Code,Indicator Name,Indicator Code,Continente,Año,GastoMilitarTotal,top15_2020
49,Alemania,DEU,Military expenditure (current USD),MS.MIL.XPND.CD,Europe,1960,2.715197e+09,6
262,Alemania,DEU,Military expenditure (current USD),MS.MIL.XPND.CD,Europe,1961,3.074753e+09,6
475,Alemania,DEU,Military expenditure (current USD),MS.MIL.XPND.CD,Europe,1962,4.055355e+09,6
688,Alemania,DEU,Military expenditure (current USD),MS.MIL.XPND.CD,Europe,1963,4.688620e+09,6
901,Alemania,DEU,Military expenditure (current USD),MS.MIL.XPND.CD,Europe,1964,4.601310e+09,6


### Plot 3 Top 15 agentes del 2020, ver su desarrollo histórico
...pero eso es otra historia

In [10]:
fig3 = px.bar(top15absoluto2020 ,x="Country Name",y="GastoMilitarTotal",color="Continente",animation_group="Country Name",animation_frame="Año" , range_y=[60000000,top15absoluto2020.GastoMilitarTotal.max()*0.5],
              title="Siguiendo el desarrollo histórico de los 15 agentes internacionales más influyentes de la actualidad, desde 1960.",
              labels={
                     "GastoMilitarTotal": "Gasto Militar Total (USD)",
                     "Country Name": "",
                 }
              )
fig3.update_layout(transition = {'duration': 1000})
fig3.show()

## Bueno, y cómo va el resto, no juegan al juego este o qué?

In [11]:
NoTop15absoluto2020 = NoTop15absoluto2020[NoTop15absoluto2020['Año'] == '2020']
NoTop15absoluto2020 = NoTop15absoluto2020.dropna()
NoTop15absoluto2020.head()

,Country Name,Country Code,Indicator Name,Indicator Code,Continente,Año,GastoMilitarTotal,top15_2020
12781,Afganistán,AFG,Military expenditure (current USD),MS.MIL.XPND.CD,Asia,2020,2.795770e+08,-1
12783,Albania,ALB,Military expenditure (current USD),MS.MIL.XPND.CD,Europe,2020,2.220227e+08,-1
12782,Angola,AGO,Military expenditure (current USD),MS.MIL.XPND.CD,Africa,2020,9.935944e+08,-1
12834,Argelia,DZA,Military expenditure (current USD),MS.MIL.XPND.CD,Africa,2020,9.708277e+09,-1
12786,Argentina,ARG,Military expenditure (current USD),MS.MIL.XPND.CD,South America,2020,2.907185e+09,-1


### Plot 4 ¿Cómo se distribuye el resto de países, quitando a los gigantes, son una balsa de aceite, todos iguales ó podemos ver algo a partir de este indicador macro-económico?

In [12]:
def plotTochoWidget(scopee = 'world', proyeccion = 'times'): #  ['africa', 'asia', 'europe', 'north america', 'south america', 'usa', 'world']
  
  fig4 = go.Figure(go.Choropleth(
      locations = NoTop15absoluto2020['Country Code'],
      locationmode = "ISO-3",
      z = NoTop15absoluto2020['GastoMilitarTotal'],
      text = NoTop15absoluto2020['Country Name'],
      colorscale = 'rainbow',  
    
      reversescale=False,
      marker_line_color='#efefef',
      marker_line_width=0.5,
      
      colorbar_title = 'Gasto Militar (USD)',
      )
  )
  
  # Establecemos las características del título y la apariencia del mapa base
  fig4.update_layout(
      title_text = 'Gasto Militar Absoluto (USD) para el año 2020, ignorando las 15 potencias armamentísticas más influyentes.',
      showlegend = False,
      geo = dict(
          scope= scopee , #AQUÍ ESTÁ LA MAGIA
          resolution=50,
          #projection_type = 'orthographic',  
          projection_type= proyeccion, # AQUÍ ESTÁ LA MAGIAx2
          showcoastlines=True, coastlinecolor="Black", 
          showocean=True,
          showcountries=True,
          oceancolor="LightBlue",  
          lakecolor="Blue",
          showland=True, landcolor="white",
          #tamaño imagen
            )
      )
  
  fig4.update_layout(height=500, margin={"r":10,"t":50,"l":10,"b":10} , width = 800)
  
  fig4.show()

#import ipywidgets as widgets
widgets.interact(plotTochoWidget, scopee = ['africa', 'asia', 'europe', 'north america', 'south america', 'world'], proyeccion = ['orthographic', 'times'] )# ['natural earth', 'miller', 'orthographic', 'times', 'airy'])


interactive(children=(Dropdown(description='scopee', index=5, options=('africa', 'asia', 'europe', 'north amer…

<function __main__.plotTochoWidget(scopee='world', proyeccion='times')>

### Plot 5~extra, mapa mundi con todos los países

In [ ]:
#gráfico extra con todos juntos, al verlo se entiende por qué no se incorpora como principal
fig5 = go.Figure(go.Choropleth(
      locations = gastoMilitarAbsoluto['Country Code'],
      locationmode = "ISO-3",
      z = NoTop15absoluto2020['GastoMilitarTotal'],
      text = NoTop15absoluto2020['Country Name'],
      colorscale = 'reds',  
    
      reversescale=False,
      marker_line_color='#efefef',
      marker_line_width=0.5,
      
      colorbar_title = 'Gasto Militar (USD)',
      )
  )
  
# Establecemos las características del título y la apariencia del mapa base
fig5.update_layout(
    title_text = 'Gasto Militar Absoluto (USD), 2020.',
    showlegend = True,
    geo = dict(
        scope= 'world' , 
        resolution=50,
        #projection_type = 'orthographic',  
        projection_type= 'orthographic', 
        showcoastlines=True, coastlinecolor="Black", 
        showocean=True,
        showcountries=True,
        oceancolor="LightBlue",  
        lakecolor="Blue",
        showland=True, landcolor="white",
        #tamaño imagen
          )
    )

fig5.update_layout(height=900, margin={"r":10,"t":50,"l":10,"b":10} , width = 1800)

fig5.show()

# Parte 2: Análisis de datos de homicidios intencionados, seguiré en excel ó datawraper

In [30]:
#funcion get columna continente y limpia paises sin sentido (que no formen parte del diccionario country_alpha2 ó 3 de pc library)
def getColumnaContinente(df):
  df=df.assign(Continente = 1) 
  def getContinents(x):
    #from alphacode3 in dataframe
      try:
          return  pc.convert_continent_code_to_continent_name(pc.country_alpha2_to_continent_code(pc.country_alpha3_to_country_alpha2(x)) )
      except:
          return 'Undetermined'
  df.Continente = df["Country Code"].apply(getContinents)
  df = df.drop(df[df['Continente'] == 'Undetermined'].index )
  return df

In [31]:
!wget https://api.worldbank.org/v2/es/indicator/SP.POP.TOTL?downloadformat=csv >/dev/null #poblacion por pais
!unzip SP.POP.TOTL?downloadformat=csv >/dev/null
!!head -n5 API_SP.POP.TOTL_DS2_es_csv_v2_4682121.csv 
#!rm *=csv*

--2022-11-22 18:28:19--  https://api.worldbank.org/v2/es/indicator/SP.POP.TOTL?downloadformat=csv
Resolving api.worldbank.org (api.worldbank.org)... 52.188.136.184
Connecting to api.worldbank.org (api.worldbank.org)|52.188.136.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80377 (78K) [application/zip]
Saving to: ‘SP.POP.TOTL?downloadformat=csv.1’

SP.POP.TOTL?downloa 100%[===================>]  78.49K  --.-KB/s    in 0.03s   

2022-11-22 18:28:19 (2.81 MB/s) - ‘SP.POP.TOTL?downloadformat=csv.1’ saved [80377/80377]

replace Metadata_Indicator_API_SP.POP.TOTL_DS2_es_csv_v2_4682121.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


['\ufeff"Data Source","Indicadores del desarrollo mundial",',
 '',
 '"Last Updated Date","2022-09-16",',
 '',
 '"Country Name","Country Code","Indicator Name","Indicator Code","1960","1961","1962","1963","1964","1965","1966","1967","1968","1969","1970","1971","1972","1973","1974","1975","1976","1977","1978","1979","1980","1981","1982","1983","1984","1985","1986","1987","1988","1989","1990","1991","1992","1993","1994","1995","1996","1997","1998","1999","2000","2001","2002","2003","2004","2005","2006","2007","2008","2009","2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020","2021",']

In [32]:
poblacionMundial = pd.read_csv("/content/API_SP.POP.TOTL_DS2_es_csv_v2_4682121.csv", skiprows = 4) # sobre 100 000 habitantes
poblacionMundial.drop(columns = 'Unnamed: 66', inplace = True)
poblacionMundial.columns
poblacionMundial = poblacionMundial[['Country Name', 'Country Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021'] ]
poblacionMundial = poblacionMundial.melt(id_vars = ['Country Name', 'Country Code'], value_name = 'population', var_name = 'Año')
poblacionMundial = getColumnaContinente(poblacionMundial)
#ffoward por pais
poblacionMundial['population'] = poblacionMundial.groupby('Country Name')['population'].ffill()

# nº nas
poblacionMundial.population.isna().sum()

30

In [33]:
!wget https://api.worldbank.org/v2/es/indicator/VC.IHR.PSRC.P5?downloadformat=csv >/dev/null # homicidios intencionados
!unzip /content/VC.IHR.PSRC.P5?downloadformat=csv >/dev/null
!head -n5 API_VC.IHR.PSRC.P5_DS2_es_csv_v2_4499705.csv

--2022-11-22 18:28:51--  https://api.worldbank.org/v2/es/indicator/VC.IHR.PSRC.P5?downloadformat=csv
Resolving api.worldbank.org (api.worldbank.org)... 52.188.136.184
Connecting to api.worldbank.org (api.worldbank.org)|52.188.136.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44776 (44K) [application/zip]
Saving to: ‘VC.IHR.PSRC.P5?downloadformat=csv.1’

VC.IHR.PSRC.P5?down 100%[===================>]  43.73K  --.-KB/s    in 0.03s   

2022-11-22 18:28:52 (1.48 MB/s) - ‘VC.IHR.PSRC.P5?downloadformat=csv.1’ saved [44776/44776]

replace Metadata_Indicator_API_VC.IHR.PSRC.P5_DS2_es_csv_v2_4499705.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
﻿"Data Source","Indicadores del desarrollo mundial",

"Last Updated Date","2022-09-15",

"Country Name","Country Code","Indicator Name","Indicator Code","1960","1961","1962","1963","1964","1965","1966","1967","1968","1969","1970","1971","1972","1973","1974","1975","1976","1977","1978","1979","1980","1981","1982","1983","198

In [34]:
homicidiosIntencionados = pd.read_csv("/content/API_VC.IHR.PSRC.P5_DS2_es_csv_v2_4499705.csv", skiprows = 4) # sobre 100 000 habitantes
homicidiosIntencionados.drop(columns = 'Unnamed: 66', inplace = True)
# the melting
homicidiosIntencionados.columns
homicidiosIntencionados = homicidiosIntencionados.melt(id_vars = ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'], value_name = 'HomicidiosPor100000', var_name = 'Año')

homicidiosIntencionados = getColumnaContinente(homicidiosIntencionados)
#ffoward por pais
homicidiosIntencionados['HomicidiosPor100000'] = homicidiosIntencionados.groupby('Country Name')['HomicidiosPor100000'].ffill()
#unir 2 datasets: poblacion y asesinatos
homicidiosIntencionados = homicidiosIntencionados.assign(Población = 1) 
homicidiosIntencionados.Población = poblacionMundial.population
homicidiosIntencionados.head()

#homicidios totales por año en un pais = (homicidios/100000 habts) * nº habitantes/100000
homicidiosIntencionados = homicidiosIntencionados.assign(HomicidiosTotalesAnual = 1) 
homicidiosIntencionados.HomicidiosTotalesAnual = homicidiosIntencionados['HomicidiosPor100000'] * homicidiosIntencionados['Población'] / 100000
#ffoward por pais
homicidiosIntencionados['HomicidiosTotalesAnual'] = homicidiosIntencionados.groupby('Country Name')['HomicidiosTotalesAnual'].ffill()

#nº nas
homicidiosIntencionados.HomicidiosTotalesAnual.isna().sum()

8264

In [35]:
homicidiosIntencionados.head(10)

,Country Name,Country Code,Indicator Name,Indicator Code,Año,HomicidiosPor100000,Continente,Población,HomicidiosTotalesAnual
0,Aruba,ABW,Homicidios intencionales (por cada 100.000 hab...,VC.IHR.PSRC.P5,1960,NaN,North America,54208.0,NaN
2,Afganistán,AFG,Homicidios intencionales (por cada 100.000 hab...,VC.IHR.PSRC.P5,1960,NaN,Asia,8996967.0,NaN
4,Angola,AGO,Homicidios intencionales (por cada 100.000 hab...,VC.IHR.PSRC.P5,1960,NaN,Africa,5454938.0,NaN
5,Albania,ALB,Homicidios intencionales (por cada 100.000 hab...,VC.IHR.PSRC.P5,1960,NaN,Europe,1608800.0,NaN
6,Andorra,AND,Homicidios intencionales (por cada 100.000 hab...,VC.IHR.PSRC.P5,1960,NaN,Europe,13410.0,NaN
8,Emiratos Árabes Unidos,ARE,Homicidios intencionales (por cada 100.000 hab...,VC.IHR.PSRC.P5,1960,NaN,Asia,92417.0,NaN
9,Argentina,ARG,Homicidios intencionales (por cada 100.000 hab...,VC.IHR.PSRC.P5,1960,NaN,South America,20481781.0,NaN
10,Armenia,ARM,Homicidios intencionales (por cada 100.000 hab...,VC.IHR.PSRC.P5,1960,NaN,Asia,1874119.0,NaN
11,Samoa Americana,ASM,Homicidios intencionales (por cada 100.000 hab...,VC.IHR.PSRC.P5,1960,NaN,Oceania,20127.0,NaN
12,Antigua y Barbuda,ATG,Homicidios intencionales (por cada 100.000 hab...,VC.IHR.PSRC.P5,1960,NaN,North America,54132.0,NaN


### Plot6 Tree-map homicidios intencionados por continente

In [36]:
df = homicidiosIntencionados.query("Año == '2020'")
df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,Año,HomicidiosPor100000,Continente,Población,HomicidiosTotalesAnual
15960,Aruba,ABW,Homicidios intencionales (por cada 100.000 hab...,VC.IHR.PSRC.P5,2020,1.927228,North America,106766.0,2.057624
15962,Afganistán,AFG,Homicidios intencionales (por cada 100.000 hab...,VC.IHR.PSRC.P5,2020,6.655561,Asia,38928341.0,2590.899541
15964,Angola,AGO,Homicidios intencionales (por cada 100.000 hab...,VC.IHR.PSRC.P5,2020,4.847075,Africa,32866268.0,1593.052718
15965,Albania,ALB,Homicidios intencionales (por cada 100.000 hab...,VC.IHR.PSRC.P5,2020,2.119675,Europe,2837849.0,60.153169
15966,Andorra,AND,Homicidios intencionales (por cada 100.000 hab...,VC.IHR.PSRC.P5,2020,2.588494,Europe,77265.0,2.000000


interactivo elegir año

In [42]:
def plotTreemapWidget(year):
  #con query eliges el año o lo que veas, recordar que el año es tipo str en mi dataframe
  year
  df = homicidiosIntencionados.query("Año == '" + year + "'")#query
  df.HomicidiosPor100000.ffill(inplace = True)
  df = df.query("Continente != 'Oceania'")#quito oceania
  fig6 = px.treemap(df, path=[px.Constant('world'), 'Continente', 'Country Name'], values='Población',
                    hover_data=['Country Code'], color='HomicidiosPor100000',
                    labels={
                "HomicidiosPor100000": "Homicidios intencionados (100.000 habts.)",
                "Country Code": "",
                'Año' : '',
                },
                color_continuous_scale='matter',    ) 
  #legend, title
  fig6.update_layout(
      title_text = f'TreeMap, muertes por cada 100000 habitantes para el año {year}',
      showlegend = True,
      )
  #color


  #size
  fig6.update_layout(height=900, margin={"r":10,"t":50,"l":10,"b":10} , width = 1500)
  fig6.show()



###Plot 6 widget por año

In [38]:
widgets.interact(plotTreemapWidget, year = reversed(list(homicidiosIntencionados.Año.unique()[30:]) ) )

interactive(children=(Dropdown(description='year', options=('2021', '2020', '2019', '2018', '2017', '2016', '2…

<function __main__.plotTreemapWidget(year)>

Gráfica suelta para el año 2020

In [39]:
#Gráfica suelta
for palette in[ 'orrd', 'matter', 'amp']:
              #con query eliges el año o lo que veas, recordar que el año es tipo str en mi dataframe
  df = homicidiosIntencionados.query("Año == '2020'")
  df.HomicidiosPor100000.ffill(inplace = True)
  df = df.query("Continente != 'Oceania'")#quito oceania
  fig6 = px.treemap(df, path=[px.Constant('world'), 'Continente', 'Country Name'], values='Población',
                    hover_data=['Country Code'], color='HomicidiosPor100000',
                    labels={
                "HomicidiosPor100000": "Homicidios intencionados (100.000 habts.)",
                "Country Code": "",
                'Año' : '',
                },
                color_continuous_scale=palette,    ) 
  #legend, title
  fig6.update_layout(
      title_text = 'TreeMap, muertes por cada 100000 habitantes',
      showlegend = True,
      )
  #color


  #size
  fig6.update_layout(height=900, margin={"r":10,"t":50,"l":10,"b":10} , width = 1500)
  fig6.show()
  print('palete anterior:', palette)


palete anterior: orrd


palete anterior: matter


palete anterior: amp


In [40]:
#posibles paletas a elegir [ 'orrd', 'matter', 'amp']
#con query eliges el año o lo que veas, recordar que el año es tipo str en mi dataframe
df = homicidiosIntencionados.query("Año == '2020'")
df.HomicidiosPor100000.ffill(inplace = True)
df = df.query("Continente != 'Oceania'")#quito oceania
fig6 = px.treemap(df, path=[px.Constant('world'), 'Continente', 'Country Name'], values='Población',
                    color='HomicidiosPor100000',
                  labels={
              "HomicidiosPor100000": "Homicidios intencionados (100.000 habts.)",
              "Country Code": "",
              'Año' : '',
              'parent':'Continente'
               },
               color_continuous_scale='matter',    ) 
#legend, title
fig6.update_layout(
    title_text = 'TreeMap, muertes por cada 100000 habitantes',
    showlegend = True,
    )
#color


#size
fig6.update_layout(height=900, margin={"r":10,"t":50,"l":10,"b":10} , width = 1500)
fig6.show()


# Guardar html


In [ ]:
import plotly
plotly.offline.plot(fig1, filename = 'fig1.html', auto_open=False)
plotly.offline.plot(fig2, filename = 'fig2.html', auto_open=False)
plotly.offline.plot(fig3, filename = 'fig3.html', auto_open=False)
plotly.offline.plot(fig5, filename = 'fig5.html', auto_open=False)
plotly.offline.plot(fig6, filename = 'fig6.html', auto_open=False)
#plotly.offline.plot(fig5, filename = 'fig3.html', auto_open=False) # no lo guardo porque es meirda
for scopee in ['africa', 'asia', 'europe', 'north america', 'south america', 'world']:
  proyeccion = 'times'
  if scopee == 'world':
    proyeccion = 'orthographic'

  fig4 = go.Figure(go.Choropleth(
        locations = NoTop15absoluto2020['Country Code'],
        locationmode = "ISO-3",
        z = NoTop15absoluto2020['GastoMilitarTotal'],
        text = NoTop15absoluto2020['Country Name'],
        colorscale = 'rainbow',  
      
        reversescale=False,
        marker_line_color='#efefef',
        marker_line_width=0.5,
        
        colorbar_title = 'Gasto Militar (USD)',
        )
    )
  
  # Establecemos las características del título y la apariencia del mapa base
  fig4.update_layout(
      title_text = f'Gasto Militar Absoluto (USD) para el año 2020, ignorando las 15 potencias armamentísticas más influyentes. <br><sup>(Vista {scopee.capitalize()})</sup>',     
      
      showlegend = False,
      geo = dict(
          scope= scopee , #AQUÍ ESTÁ LA MAGIA
          resolution=50,
          #projection_type = 'orthographic',  
          projection_type= proyeccion, # AQUÍ ESTÁ LA MAGIAx2
          showcoastlines=True, coastlinecolor="Black", 
          showocean=True,
          showcountries=True,
          oceancolor="LightBlue",  
          lakecolor="Blue",
          showland=True, landcolor="white",
      )
  )
  fig4.update_layout(height=900, margin={"r":10,"t":50,"l":10,"b":10}, width= 1800)
  fig4.show()
  plotly.offline.plot(fig4, filename = 'fig4'+scopee+'.html', auto_open=False)


In [ ]:
!mkdir HTMLFigurespython
!mv ./fig* HTMLFigurespython/
!mv HTMLFigurespython /content/drive/MyDrive/MBID

### Exportar csv asesinatos


In [ ]:
homicidiosIntencionados2020 = homicidiosIntencionados.ffill()
homicidiosIntencionados2020 = homicidiosIntencionados2020.bfill()
homicidiosIntencionados2020 = homicidiosIntencionados.query("Año == '2020'")

homicidiosIntencionados2000 = homicidiosIntencionados.ffill()
homicidiosIntencionados2000 = homicidiosIntencionados2000.bfill()
homicidiosIntencionados2000 = homicidiosIntencionados.query("Año == '2000'")

homicidiosIntencionados2020.to_csv("homicidiosIntencionados2020.csv")
homicidiosIntencionados2000.to_csv("homicidiosIntencionados2000.csv")

# Cómo incorporar gráficas de datawraper en htmls de pyplot

<iframe title="Distribución mundial del Nº de Homicidios Violentos por 100.000 habitantes en el año 2020" aria-label="Map" id="datawrapper-chart-Cljmz" src="https://datawrapper.dwcdn.net/Cljmz/1/" scrolling="no" frameborder="0" style="width: 0; min-width: 100% !important; border: none;" height="633" data-external="1"></iframe><script type="text/javascript">!function(){"use strict";window.addEventListener("message",(function(e){if(void 0!==e.data["datawrapper-height"]){var t=document.querySelectorAll("iframe");for(var a in e.data["datawrapper-height"])for(var r=0;r<t.length;r++){if(t[r].contentWindow===e.source)t[r].style.height=e.data["datawrapper-height"][a]+"px"}}}))}();
</script>
incluye este script en el body y a gozar

## limpieza de datos de suicidios datosmacro

In [26]:
import plotly.express as px
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pycountry_convert as pc
import ipywidgets as widgets
import plotly.graph_objects as go
#read several sheets of excel -> https://stackoverflow.com/questions/26521266/using-pandas-to-pd-read-excel-for-multiple-worksheets-of-the-same-workbook


In [27]:
suicidiosDict = pd.ExcelFile('/content/drive/MyDrive/MBID/suicidioMundo2020_2006.xlsx')
suicidios2006 = pd.read_excel(suicidiosDict,'2006',  skiprows = 1, header = 0)
suicidios2008 = pd.read_excel(suicidiosDict,'2008', skiprows = 1, header = 0, )
suicidios2010 = pd.read_excel(suicidiosDict, '2010', skiprows = 1, header = 0)#, '2010')
suicidios2012 = pd.read_excel(suicidiosDict, '2012', skiprows = 1, header = 0)#, '2012')
suicidios2014 = pd.read_excel(suicidiosDict, '2014', skiprows = 1, header = 0)#, '2014')
suicidios2016 = pd.read_excel(suicidiosDict, '2016', skiprows = 1, header = 0)#, '2016')
suicidios2018 = pd.read_excel(suicidiosDict, '2018', skiprows = 1, header = 0)#, '2018')
suicidios2020 = pd.read_excel(suicidiosDict, '2020', skiprows = 1, header = 0)#, '2020')
display(suicidios2006.head() )
display(suicidios2006.columns )

,Unnamed: 0,Unnamed: 1,Países,Fecha,Suicidios mujeres,Suicidios hombres,Suicidios,Suicidios tasa femenina,Suicidios tasa masculina,Suicidios por 100.000,Var.
0,NaN,NaN,España [+],2006,734,2512,3246,3.29,11.56,7.37,-0.0595
1,NaN,NaN,Alemania [+],2006,2543,7231,9774,6.00,17.90,11.90,-0.0403
2,NaN,NaN,Reino Unido [+],2006,936,3186,4122,3.00,10.70,6.80,0.0149
3,NaN,NaN,Francia [+],2006,2866,7744,10610,8.80,25.20,16.70,-0.0347
4,NaN,NaN,Italia [+],2006,859,2842,3701,2.80,9.90,6.30,-0.0308


Index(['Unnamed: 0', 'Unnamed: 1', 'Países', 'Fecha', 'Suicidios mujeres',
       'Suicidios hombres', 'Suicidios ', 'Suicidios tasa femenina',
       'Suicidios tasa masculina', 'Suicidios por 100.000', 'Var.'],
      dtype='object')

In [28]:
#poblacionMundial = poblacionMundial.melt(id_vars = ['Country Name', 'Country Code'],
 #                                        value_name = 'population', var_name = 'Año')
#ffoward por pais
#poblacionMundial['population'] = poblacionMundial.groupby('Country Name')['population'].ffill()


def funcionSucidios(df, anio, toDrop=[]):
  df.Países = df.Países.str.replace('+', '')
  df.Países = df.Países.str.replace(']', '')
  df.Países = df.Países.str.replace('[', '')
  df.Países = df.Países.str.strip()
  df = df.assign(Año = anio)
  for elemento in toDrop:
    try:
      df.drop(columns =elemento, inplace = True)
    except:
      pass
  print(f'columnas del nuevo dataframe: {df.columns}')
  return df

suicidios2006 = funcionSucidios(suicidios2006, 2006, ['Fecha', 'Var.','Unnamed: 0', 'Unnamed: 1'])
suicidios2008 = funcionSucidios(suicidios2008, 2008, ['Fecha', 'Var.','Unnamed: 0', 'Unnamed: 1'])
suicidios2010 = funcionSucidios(suicidios2010, 2010, ['Fecha', 'Var.','Unnamed: 0', 'Unnamed: 1'])
suicidios2012 = funcionSucidios(suicidios2012, 2012, ['Fecha', 'Var.','Unnamed: 0', 'Unnamed: 1'])
suicidios2014 = funcionSucidios(suicidios2014, 2014, ['Fecha', 'Var.','Unnamed: 0', 'Unnamed: 1'])
suicidios2016 = funcionSucidios(suicidios2016, 2016, ['Fecha', 'Var.','Unnamed: 0', 'Unnamed: 1'])
suicidios2018 = funcionSucidios(suicidios2018, 2018, ['Fecha', 'Var.','Unnamed: 0', 'Unnamed: 1'])
suicidios2020 = funcionSucidios(suicidios2020, 2020, ['Fecha', 'Var.','Unnamed: 0', 'Unnamed: 1'])

display(suicidios2006.head() )
display(suicidios2020.head() )

columnas del nuevo dataframe: Index(['Países', 'Suicidios mujeres', 'Suicidios hombres', 'Suicidios ',
       'Suicidios tasa femenina', 'Suicidios tasa masculina',
       'Suicidios por 100.000', 'Año'],
      dtype='object')
columnas del nuevo dataframe: Index(['Países', 'Suicidios mujeres', 'Suicidios hombres', 'Suicidios ',
       'Suicidios tasa femenina', 'Suicidios tasa masculina',
       'Suicidios por 100.000', 'Año'],
      dtype='object')
columnas del nuevo dataframe: Index(['Países', 'Suicidios mujeres', 'Suicidios hombres', 'Suicidios ',
       'Suicidios tasa femenina', 'Suicidios tasa masculina',
       'Suicidios por 100.000', 'Año'],
      dtype='object')
columnas del nuevo dataframe: Index(['Países', 'Suicidios mujeres', 'Suicidios hombres', 'Suicidios ',
       'Suicidios tasa femenina', 'Suicidios tasa masculina',
       'Suicidios por 100.000', 'Año'],
      dtype='object')
columnas del nuevo dataframe: Index(['Países', 'Suicidios mujeres', 'Suicidios hombres', 'Su

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



,Países,Suicidios mujeres,Suicidios hombres,Suicidios,Suicidios tasa femenina,Suicidios tasa masculina,Suicidios por 100.000,Año
0,España,734,2512,3246,3.29,11.56,7.37,2006
1,Alemania,2543,7231,9774,6.00,17.90,11.90,2006
2,Reino Unido,936,3186,4122,3.00,10.70,6.80,2006
3,Francia,2866,7744,10610,8.80,25.20,16.70,2006
4,Italia,859,2842,3701,2.80,9.90,6.30,2006


,Países,Suicidios mujeres,Suicidios hombres,Suicidios,Suicidios tasa femenina,Suicidios tasa masculina,Suicidios por 100.000,Año
0,España,1011,2930,3941,4.20,12.60,8.30,2020
1,Alemania,2268,7015,9283,5.40,16.90,11.10,2020
2,Reino Unido,1337,4187,5524,3.90,12.60,8.20,2020
3,Francia,2091,6298,8389,6.04,19.45,12.53,2020
4,Italia,824,2901,3725,2.65,9.78,6.12,2020


In [41]:
!jupyter nbconvert /content/drive/MyDrive/MBID/MBID08PlotlyJavierBescos.ipynb --no-input --no-prompt

[NbConvertApp] Converting notebook /content/drive/MyDrive/MBID/MBID08PlotlyJavierBescos.ipynb to html
[NbConvertApp] Writing 1924591 bytes to /content/drive/MyDrive/MBID/MBID08PlotlyJavierBescos.html


# para cuando incorpores las imagenes del data wraper
https://es.stackoverflow.com/questions/77634/como-agregar-una-imagen-a-un-notebook-en-jupyter

https://es.stackoverflow.com/questions/443590/ocultar-c%C3%B3digo-y-n%C3%BAmero-de-input-y-output-en-jupyter-al-exportarlo-como-pdf-via
y el 2o para cuando pases de notebook a pdf(ocultando código)
